# Extract Data from FoodData Central API (Free to use)

## Using pandas and request

In [ ]:
Install request:

In [ ]:
!pip install requests

In [ ]:
import requests
print(requests.__version__)

Install duckdb

In [ ]:
!pip install duckdb

In [ ]:
import duckdb
print(duckdb.__version__)

Before running this code, please sign up to get the api key on this website <br> https://fdc.nal.usda.gov/api-guide#bkmk-3 <br> <h3> Gaining Access </h3>
Anyone may access and use the API. However, a data.gov API key must be incorporated into each API request. Sign up to obtain a key, then follow the instructions for how to use your key.

### Get all raw data from API

In [ ]:
import requests
import pandas as pd
import duckdb

# Your USDA API key
API_KEY = 'insert_your_api_key'

# Define search query
search_query = "chicken breast"

# Step 1: Make the request to search endpoint
search_url = f'https://api.nal.usda.gov/fdc/v1/foods/search'
params = {
    'query': search_query,
    'pageSize': 15,  # small sample
    'api_key': API_KEY
}

response = requests.get(search_url, params=params)
data = response.json()

# Step 2: Extract relevant fields into a flat table
foods = data.get("foods", [])
records = []
for food in foods:
    records.append({
        'fdcId': food.get('fdcId'),
        'description': food.get('description'),
        'brandOwner': food.get('brandOwner'),
        'dataType': food.get('dataType'),
        'foodCategory': food.get('foodCategory'),
        'ingredients': food.get('ingredients'),
        'calories': next((n.get('value') for n in food.get('foodNutrients', []) if n.get('nutrientName') == 'Energy'), None),
        'protein': next((n.get('value') for n in food.get('foodNutrients', []) if n.get('nutrientName') == 'Protein'), None),
        'carbs': next((n.get('value') for n in food.get('foodNutrients', []) if n.get('nutrientName') == 'Carbohydrate, by difference'), None),
        'fat': next((n.get('value') for n in food.get('foodNutrients', []) if n.get('nutrientName') == 'Total lipid (fat)'), None)
    })

# Step 3: Create DataFrame
df = pd.DataFrame(records)

# Step 4: Store into DuckDB
con = duckdb.connect("usda_data.duckdb")
con.execute("CREATE TABLE IF NOT EXISTS food_data AS SELECT * FROM df")

# Step 5: Query from DuckDB and show result
result_df = con.execute("SELECT * FROM food_data").fetchdf()
print(result_df.head(15))

Make the output more clean and readable

In [ ]:
# Set Pandas Display Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)  # Increase column width for long text
pd.set_option('display.width', 120)         # Wider print layout

In [ ]:
# Clean ingredients Column, truncate to the first 80 characters
df['ingredients'] = df['ingredients'].str.slice(0, 80) + '...'

In [ ]:
# Round Nutrient Values
df[['calories', 'protein', 'carbs', 'fat']] = df[['calories', 'protein', 'carbs', 'fat']].round(1)

In [ ]:
# Reorder & Rename Columns for Clarity
df_clean = df[[
    'fdcId', 'description', 'brandOwner', 'foodCategory',
    'calories', 'protein', 'carbs', 'fat', 'ingredients'
]].rename(columns={
    'fdcId': 'FDC ID',
    'description': 'Description',
    'brandOwner': 'Brand',
    'foodCategory': 'Category',
    'calories': 'Calories',
    'protein': 'Protein (g)',
    'carbs': 'Carbs (g)',
    'fat': 'Fat (g)',
    'ingredients': 'Ingredients'
})

In [ ]:
print(df_clean.head(15).to_string(index=False))

<h3> Download DataFrame as CSV </h3>
Use encoding parameter to specify the encoding format, such as 'utf-8', to handle special characters properly

In [ ]:
df_clean.to_csv('sample_data_from_USDA.csv', encoding='utf-8', index=False)

### Normalize the nutrients into a separate table, where:

foods table contains basic metadata like description, brand, etc.
nutrients table contains each nutrient value per food item, in a long-form layout (one row per food x nutrient combination).

In [ ]:
import requests
import pandas as pd
import duckdb

# --- API config ---
API_KEY = 'insert_your_api_key'
search_query = "chicken breast"
page_size = 15

# -- SEARCH USDA API
search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
params = {
    "query": search_query,
    "pageSize": page_size,
    "api_key": API_KEY
}

response = requests.get(search_url, params=params)
data = response.json()
foods = data.get("foods", [])

# -- NORMALIZE: FOOD ITEM METADATA TABLE
food_metadata = []
nutrient_records = []

for food in foods:
    fdcId = food.get("fdcId")

    food_metadata.append({
        "fdcId": fdcId,
        "description": food.get("description"),
        "brandOwner": food.get("brandOwner"),
        "dataType": food.get("dataType"),
        "foodCategory": food.get("foodCategory"),
        "ingredients": food.get("ingredients")
    })

    for nutrient in food.get("foodNutrients", []):
        nutrient_records.append({
            "fdcId": fdcId,
            "nutrientName": nutrient.get("nutrientName"),
            "nutrientUnit": nutrient.get("unitName"),
            "nutrientValue": nutrient.get("value")
        })

# -- CREATE DATAFRAMES
df_food = pd.DataFrame(food_metadata)
df_nutrients = pd.DataFrame(nutrient_records)

# -- OPTIONAL: Clean up ingredient text
df_food['ingredients'] = df_food['ingredients'].astype(str).str.slice(0, 100) + '...'

# -- SAVE TO CSV
df_food.to_csv("food_items.csv", index=False)

# -- Display samples
print("Food Items Table:")
print(df_food.head(15).to_string(index=False))

In [ ]:
# -- SAVE TO CSV
df_nutrients.to_csv("food_nutrients.csv", index=False)

# -- Display samples
print("Nutrients Table:")
print(df_nutrients.head(15).to_string(index=False))

# Extract Data from Spoonacular API (Not recommended - no free edition anymore)

Before running this code, please sign up to get the api key on this website <br> https://spoonacular.com/food-api/docs

## Using pandas and request

### Get all raw data from API

In [ ]:
import requests
import pandas as pd
import duckdb

API_KEY = "YOUR_SPOONACULAR_API_KEY"

# Clean minimal recipe input
ingredient_list = """2 eggs
1 cup flour
1 cup milk
1 tsp salt
1 tbsp sugar
"""

url = "https://api.spoonacular.com/recipes/analyze"
params = {"apiKey": API_KEY}
headers = {"Content-Type": "application/x-www-form-urlencoded"}
data = {
    "title": "Pancake Recipe",
    "ingredientList": ingredient_list,
    "servings": 3
}

# Send POST request
response = requests.post(url, headers=headers, params=params, data=data)

# Print error if any
if response.status_code != 200:
    print(f"Error {response.status_code}:")
    print(response.text)
    exit()

parsed = response.json()

# Parse recipe summary
recipe_info = {
    "title": parsed.get("title"),
    "servings": parsed.get("servings"),
    "calories": parsed.get("nutrition", {}).get("nutrients", [{}])[0].get("amount"),
    "calories_unit": parsed.get("nutrition", {}).get("nutrients", [{}])[0].get("unit"),
    "source": "Spoonacular"
}
df_recipe = pd.DataFrame([recipe_info])

# Parse ingredients
ingredient_list = []
for ing in parsed.get("ingredients", []):
    ingredient_list.append({
        "name": ing.get("name"),
        "amount": ing.get("amount"),
        "unit": ing.get("unit"),
        "original": ing.get("original"),
        "categoryPath": ", ".join(ing.get("categoryPath", [])) if ing.get("categoryPath") else ""
    })
df_ingredients = pd.DataFrame(ingredient_list)

# Save to DuckDB
con = duckdb.connect("recipes.duckdb")
con.execute("CREATE OR REPLACE TABLE recipes AS SELECT * FROM df_recipe")
con.execute("CREATE OR REPLACE TABLE ingredients AS SELECT * FROM df_ingredients")

# Show results
print("*  Recipe Info:")
print(df_recipe.to_string(index=False))

print("\n* Ingredients:")
print(df_ingredients.to_string(index=False))

# Extract Data from Edamam API (not recommended - free edition always show 404)

Before running this code, please sign up to get the api key on this website
https://www.edamam.com/

## Using pandas and request

### Get all raw data (meal plan) from API

In [ ]:
import requests
import pandas as pd
import duckdb

In [ ]:
# --- Set your Edamam Meal Planner API credentials
APP_ID = "your_app_id"
APP_KEY = "your_app_key"

# --- Endpoint URL
url = "https://api.edamam.com/api/meal-planner/v1"

# --- Example payload
payload = {
    "calendar": {
        "calorieCount": 2000,
        "diet": "balanced",     # e.g., "low-carb", "high-protein"
        "numMeals": 3,
        "timeFrame": "week"
    }
}

# --- Send POST request
response = requests.post(
    url,
    params={"app_id": APP_ID, "app_key": APP_KEY},
    json=payload
)

# --- Check response status
if response.status_code != 200:
    print("Error:", response.status_code)
    print(response.text)
    response.raise_for_status()

# --- Parse JSON response
data = response.json()
meals = data.get("calendar", {}).get("meals", [])

# --- Flatten meals into DataFrame
meal_records = []
for day in meals:
    for meal in day.get("meals", []):
        meal_records.append({
            "day": day.get("day"),
            "meal_type": meal.get("mealType"),
            "title": meal.get("label"),
            "calories": meal.get("calories"),
            "recipe_url": meal.get("url")
        })

df_meals = pd.DataFrame(meal_records)

# --- Save to DuckDB
con = duckdb.connect("mealplanner.duckdb")
con.execute("CREATE OR REPLACE TABLE meal_plan AS SELECT * FROM df_meals")

# --- Show sample output
print("Weekly Meal Plan:")
print(df_meals.head().to_string(index=False))

### Get all raw data (recipe plan) from API

In [ ]:
import requests
import pandas as pd
import duckdb

# Replace with your Edamam credentials
APP_ID = "your_real_app_id"
APP_KEY = "your_real_app_key"

query = "chicken"
url = "https://api.edamam.com/api/recipes/v2"
params = {
    "type": "public",
    "q": query,
    "app_id": APP_ID,
    "app_key": APP_KEY,
    "from": 0,
    "to": 5
}

# API request
response = requests.get(url, params=params)

# Check if credentials work
if response.status_code != 200:
    print(" API Error:", response.status_code)
    print(" Hint:", response.text)
    response.raise_for_status()

# Parse and load
data = response.json()
hits = data.get("hits", [])

recipes = []
for hit in hits:
    r = hit["recipe"]
    recipes.append({
        "label": r.get("label"),
        "source": r.get("source"),
        "calories": r.get("calories"),
        "servings": r.get("yield"),
        "ingredients": "; ".join(r.get("ingredientLines", [])),
        "url": r.get("url")
    })

df = pd.DataFrame(recipes)

# Save to DuckDB
con = duckdb.connect("recipes.duckdb")
con.execute("CREATE OR REPLACE TABLE recipes AS SELECT * FROM df")

print("Sample Recipes:")
print(df.head().to_string(index=False))

# Extract Data from themealdb (Free to use)

## Using pandas and request

### Get all raw data from API

In [ ]:
import requests
import pandas as pd
import duckdb

In [ ]:
# Step 1: Fetch data from TheMealDB API
# Example: Search for meals with "chicken"
url = "https://www.themealdb.com/api/json/v1/1/search.php?s=chicken"
response = requests.get(url)

# Check for errors
if response.status_code != 200:
    print(f"API Error {response.status_code}")
    print(response.text)
    response.raise_for_status()

# Parse JSON
data = response.json()
meals = data.get("meals", [])

# Step 2: Normalize meals into flat records
records = []
for meal in meals:
    records.append({
        "meal_id": meal.get("idMeal"),
        "meal_name": meal.get("strMeal"),
        "category": meal.get("strCategory"),
        "area": meal.get("strArea"),
        "instructions": meal.get("strInstructions"),
        "meal_thumb": meal.get("strMealThumb"),
        "tags": meal.get("strTags"),
        "youtube": meal.get("strYoutube"),
        "ingredients": "; ".join(
            [f"{meal.get(f'strMeasure{i}', '').strip()} {meal.get(f'strIngredient{i}', '').strip()}"
             for i in range(1, 21)
             if meal.get(f'strIngredient{i}') and meal.get(f'strIngredient{i}').strip()]
        )
    })

# Step 3: Create Pandas DataFrame
df = pd.DataFrame(records)

# Step 4: Save to DuckDB
con = duckdb.connect("meals.duckdb")
con.execute("CREATE OR REPLACE TABLE meals AS SELECT * FROM df")

# Step 5: Preview output
print("Sample Recipes from TheMealDB:")
print(df.head().to_string(index=False))

### Normalize the data by separating meals and ingredients into two related tables:

In [ ]:
# --- Step 1: Fetch data from TheMealDB
url = "https://www.themealdb.com/api/json/v1/1/search.php?s=chicken"
response = requests.get(url)

# Check status
if response.status_code != 200:
    print(f"Error {response.status_code}")
    print(response.text)
    response.raise_for_status()

data = response.json()
meals = data.get("meals", [])

# --- Step 2: Normalize into two tables: meals & ingredients
meal_records = []
ingredient_records = []

for meal in meals:
    meal_id = meal.get("idMeal")

    # Meal table
    meal_records.append({
        "meal_id": meal_id,
        "meal_name": meal.get("strMeal"),
        "category": meal.get("strCategory"),
        "area": meal.get("strArea"),
        "instructions": meal.get("strInstructions"),
        "tags": meal.get("strTags"),
        "meal_thumb": meal.get("strMealThumb"),
        "youtube": meal.get("strYoutube")
    })

    # Ingredients table (normalized: 1 row per ingredient)
    for i in range(1, 21):  # Up to 20 ingredients
        ingredient = meal.get(f"strIngredient{i}")
        measure = meal.get(f"strMeasure{i}")
        if ingredient and ingredient.strip():
            ingredient_records.append({
                "meal_id": meal_id,
                "ingredient": ingredient.strip(),
                "measure": measure.strip() if measure else None
            })

# --- Step 3: Create DataFrames
df_meals = pd.DataFrame(meal_records)
df_ingredients = pd.DataFrame(ingredient_records)

# --- Step 4: Save to DuckDB
con = duckdb.connect("themealdb_normalized.duckdb")
con.execute("CREATE OR REPLACE TABLE meals AS SELECT * FROM df_meals")
con.execute("CREATE OR REPLACE TABLE ingredients AS SELECT * FROM df_ingredients")

# --- Step 5: Preview both tables
print("Meals Table:")
print(df_meals.head().to_string(index=False))

In [ ]:
print("\n Ingredients Table:")
print(df_ingredients.head(10).to_string(index=False))

<h3> Download DataFrame as CSV </h3>
Use encoding parameter to specify the encoding format, such as 'utf-8', to handle special characters properly

In [ ]:
# --- Save meals table to CSV
df_meals.to_csv("meals_sample.csv", index=False)
print(" Saved meals to meals_sample.csv")

# --- Save ingredients table to CSV
df_ingredients.to_csv("ingredients_sample.csv", index=False)
print(" Saved ingredients to ingredients_sample.csv")

# Extract Data from Nutritionix API (Free to use)

Before running this code, please sign up to get the api key on this website
https://developer.nutritionix.com/signup

## Using pandas and request

### Get all raw data from API

In [ ]:
import requests
import pandas as pd
import duckdb

In [ ]:
# Replace with your Nutritionix credentials
APP_ID = "your_app_id"
APP_KEY = "your_app_key"

# Natural Language Query
query = "2 eggs and a cup of rice"

# API Endpoint & Headers
url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
headers = {
    "x-app-id": APP_ID,
    "x-app-key": APP_KEY,
    "Content-Type": "application/json"
}
payload = {
    "query": query,
    "timezone": "Asia/Jakarta"
}

# Make POST request
response = requests.post(url, headers=headers, json=payload)

# Check for errors
if response.status_code != 200:
    print(f"API Error {response.status_code}")
    print(response.text)
    response.raise_for_status()

# Parse JSON response
data = response.json()
foods = data.get("foods", [])

# Normalize into flat records
records = []
for food in foods:
    records.append({
        "food_name": food.get("food_name"),
        "brand_name": food.get("brand_name"),
        "serving_qty": food.get("serving_qty"),
        "serving_unit": food.get("serving_unit"),
        "serving_weight_grams": food.get("serving_weight_grams"),
        "calories": food.get("nf_calories"),
        "total_fat": food.get("nf_total_fat"),
        "protein": food.get("nf_protein"),
        "total_carbohydrate": food.get("nf_total_carbohydrate"),
        "sugars": food.get("nf_sugars")
    })

# Create pandas DataFrame
df = pd.DataFrame(records)

# Save to DuckDB
con = duckdb.connect("nutritionix_data.duckdb")
con.execute("CREATE OR REPLACE TABLE nutrition AS SELECT * FROM df")

# Show preview
print("Nutritionix Natural Language Query Result:")
print(df.to_string(index=False))

In [ ]:
# Create pandas DataFrame (from parsed API response)
df = pd.DataFrame(records)

<h3> Download DataFrame as CSV </h3>
Use encoding parameter to specify the encoding format, such as 'utf-8', to handle special characters properly

In [ ]:
# Save the data to a CSV file
csv_file_name = "nutritionix_nutrition_data.csv"
df.to_csv(csv_file_name, index=False)

print(f"Nutrition data saved to {csv_file_name}")

# Combine All data into 1 csv file

Data Sources:
- API Ninja: Natural language food nutrition
- Nutritionix API: Nutrient breakdown via POST
- TheMealDB API: Global recipes and ingredients

## v1 - template data (small)

In [ ]:
import requests
import pandas as pd

In [ ]:
# === API CREDENTIALS ===
NINJA_API_KEY = "your_api_ninja_key"
NUTRITIONIX_APP_ID = "your_nutritionix_app_id"
NUTRITIONIX_APP_KEY = "your_nutritionix_app_key"

# === USER QUERY ===
food_query = "2 eggs and 1 cup of rice"

# === 1. API NINJAS: Nutrition Info ===
ninja_resp = requests.get(
    "https://api.api-ninjas.com/v1/nutrition",
    headers={"X-Api-Key": NINJA_API_KEY},
    params={"query": food_query}
)
ninja_data = pd.DataFrame(ninja_resp.json())

# Select key nutrition fields
df_nutrition_ninja = ninja_data[[
    "name", "serving_size_g", "calories", "protein_g", "carbohydrates_total_g", "fat_total_g", "fiber_g"
]]

# === 2. NUTRITIONIX: Nutrition Info ===
nutri_resp = requests.post(
    "https://trackapi.nutritionix.com/v2/natural/nutrients",
    headers={
        "x-app-id": NUTRITIONIX_APP_ID,
        "x-app-key": NUTRITIONIX_APP_KEY,
        "Content-Type": "application/json"
    },
    json={"query": food_query}
)
nutri_data = nutri_resp.json().get("foods", [])
df_nutrition_nutritionix = pd.DataFrame([{
    "name": f["food_name"],
    "serving_size_g": f.get("serving_weight_grams"),
    "calories": f.get("nf_calories"),
    "protein_g": f.get("nf_protein"),
    "carbohydrates_total_g": f.get("nf_total_carbohydrate"),
    "fat_total_g": f.get("nf_total_fat"),
    "fiber_g": f.get("nf_dietary_fiber")
} for f in nutri_data])

# === 3. THEMEALDB: Recipe Info ===
mealdb_resp = requests.get("https://www.themealdb.com/api/json/v1/1/search.php?s=chicken")
meals = mealdb_resp.json().get("meals", [])

# Recipe table and Ingredient table
recipes = []
ingredients = []

for meal in meals:
    meal_id = meal.get("idMeal")
    recipes.append({
        "title": meal.get("strMeal"),
        "description": f"{meal.get('strMeal')} from {meal.get('strArea')} - {meal.get('strCategory')}",
        "servings": 1,  # Placeholder, TheMealDB does not include servings
        "cook_time_minutes": None,
        "ingredients": "; ".join([
            f"{meal.get(f'strMeasure{i}', '').strip()} {meal.get(f'strIngredient{i}', '').strip()}"
            for i in range(1, 21)
            if meal.get(f'strIngredient{i}') and meal.get(f'strIngredient{i}').strip()
        ]),
        "instructions": meal.get("strInstructions"),
        "nutrition": None,
        "tags": meal.get("strTags")
    })
    for i in range(1, 21):
        ing = meal.get(f"strIngredient{i}")
        qty = meal.get(f"strMeasure{i}")
        if ing and ing.strip():
            ingredients.append({
                "name": ing.strip(),
                "quantity": qty.strip() if qty else "",
                "category": meal.get("strCategory")
            })

df_recipe = pd.DataFrame(recipes)
df_ingredient = pd.DataFrame(ingredients)

# === 4. Combine Nutrition Info ===
df_nutrition = pd.concat([df_nutrition_ninja, df_nutrition_nutritionix], ignore_index=True)

# === 5. Save to Excel with 3 Sheets ===
with pd.ExcelWriter("combined_nutrition_recipes.xlsx", engine="openpyxl") as writer:
    df_nutrition.to_excel(writer, sheet_name="NutritionInfo", index=False)
    df_ingredient.to_excel(writer, sheet_name="Ingredient", index=False)
    df_recipe.to_excel(writer, sheet_name="Recipe", index=False)

print("Excel file created: combined_nutrition_recipes.xlsx")

## v2 - more data generate

In [ ]:
import requests
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
# === API CREDENTIALS ===
NINJA_API_KEY = "your_api_ninja_key"
NUTRITIONIX_APP_ID = "your_nutritionix_app_id"
NUTRITIONIX_APP_KEY = "your_nutritionix_app_key"

# === Sample Food Queries ===
food_queries = [
    "1 apple", "1 banana", "1 cup rice", "2 eggs", "1 avocado",
    "100g chicken breast", "1 slice bread", "1 cup milk", "1 tbsp peanut butter",
    "1 cup broccoli", "1 orange", "1 cup coffee", "1 cup spinach", "1 cup yogurt"
]

# === 1. Nutrition API Ninja ===
nutrition_ninja = []
for query in food_queries:
    r = requests.get(
        "https://api.api-ninjas.com/v1/nutrition",
        headers={"X-Api-Key": NINJA_API_KEY},
        params={"query": query}
    )
    if r.status_code == 200:
        nutrition_ninja.extend(r.json())
    sleep(1)

df_ninja = pd.DataFrame(nutrition_ninja)[
    ["name", "serving_size_g", "calories", "protein_g", "carbohydrates_total_g", "fat_total_g", "fiber_g"]
]

# === 2. Nutritionix API ===
nutritionix_records = []
for query in food_queries:
    r = requests.post(
        "https://trackapi.nutritionix.com/v2/natural/nutrients",
        headers={
            "x-app-id": NUTRITIONIX_APP_ID,
            "x-app-key": NUTRITIONIX_APP_KEY,
            "Content-Type": "application/json"
        },
        json={"query": query}
    )
    if r.status_code == 200:
        for f in r.json().get("foods", []):
            nutritionix_records.append({
                "name": f["food_name"],
                "serving_size_g": f.get("serving_weight_grams"),
                "calories": f.get("nf_calories"),
                "protein_g": f.get("nf_protein"),
                "carbohydrates_total_g": f.get("nf_total_carbohydrate"),
                "fat_total_g": f.get("nf_total_fat"),
                "fiber_g": f.get("nf_dietary_fiber")
            })
    sleep(1)

df_nutritionix = pd.DataFrame(nutritionix_records)

# === Combine Nutrition Data ===
df_nutrition = pd.concat([df_ninja, df_nutritionix], ignore_index=True).drop_duplicates().head(100)

# === 3. TheMealDB Recipes & Ingredients ===
recipe_list = []
ingredient_list = []

for keyword in ["chicken", "beef", "rice", "vegetable", "egg", "pasta"]:
    r = requests.get(f"https://www.themealdb.com/api/json/v1/1/search.php?s={keyword}")
    meals = r.json().get("meals", [])
    for meal in meals:
        recipe_list.append({
            "title": meal.get("strMeal"),
            "description": f"{meal.get('strMeal')} from {meal.get('strArea')} - {meal.get('strCategory')}",
            "servings": 1,
            "cook_time_minutes": None,
            "ingredients": "; ".join([
                f"{meal.get(f'strMeasure{i}', '').strip()} {meal.get(f'strIngredient{i}', '').strip()}"
                for i in range(1, 21)
                if meal.get(f'strIngredient{i}') and meal.get(f'strIngredient{i}').strip()
            ]),
            "instructions": meal.get("strInstructions"),
            "nutrition": meal.get("nutrition"),  # May be None or missing
            "tags": meal.get("strTags")
        })

        for i in range(1, 21):
            ing = meal.get(f"strIngredient{i}")
            qty = meal.get(f"strMeasure{i}")
            if ing and ing.strip():
                ingredient_list.append({
                    "name": ing.strip(),
                    "quantity": qty.strip() if qty else "",
                    "category": meal.get("strCategory")
                })

    sleep(1)

df_recipe = pd.DataFrame(recipe_list).drop_duplicates().head(100)
df_ingredient = pd.DataFrame(ingredient_list).drop_duplicates().head(100)

# === Clean Recipe Data ===
df_recipe["nutrition"] = df_recipe["nutrition"].apply(
    lambda x: np.random.randint(1, 1001) if pd.isna(x) or str(x).strip() == "" else x
)
df_recipe.dropna(subset=["title", "ingredients", "instructions"], inplace=True)
df_recipe = df_recipe[
    (df_recipe["title"].str.strip() != "") &
    (df_recipe["ingredients"].str.strip() != "") &
    (df_recipe["instructions"].str.strip() != "")
].reset_index(drop=True)

# === 4. Save as Excel with 3 Sheets ===
with pd.ExcelWriter("combined_nutrition_recipes_final.xlsx", engine="openpyxl") as writer:
    df_nutrition.to_excel(writer, sheet_name="NutritionInfo", index=False)
    df_ingredient.to_excel(writer, sheet_name="Ingredient", index=False)
    df_recipe.to_excel(writer, sheet_name="Recipe", index=False)

print("File saved: combined_nutrition_recipes_final.xlsx")

<h3> Download DataFrame as CSV </h3>
Use encoding parameter to specify the encoding format, such as 'utf-8', to handle special characters properly

In [ ]:
# === Combine All Data (align columns)
df_all = pd.concat([df_nutrition, df_ingredient, df_recipe], ignore_index=True)
# === Save to ONE CSV ===
df_all.to_csv("combined_nutrition_recipes.csv", index=False)
print("File saved: combined_nutrition_recipes.csv")

# Summarize data into 3 csv file

In [ ]:
import requests
import pandas as pd
import numpy as np
from time import sleep

In [ ]:
# === Sample Food Queries ===
food_queries = [
    "1 apple", "1 banana", "1 cup rice", "2 eggs", "1 avocado",
    "100g chicken breast", "1 slice bread", "1 cup milk", "1 tbsp peanut butter",
    "1 cup broccoli", "1 orange", "1 cup coffee", "1 cup spinach", "1 cup yogurt"
]

# === 1. Nutrition API Ninja ===
nutrition_ninja = []
for query in food_queries:
    r = requests.get(
        "https://api.api-ninjas.com/v1/nutrition",
        headers={"X-Api-Key": NINJA_API_KEY},
        params={"query": query}
    )
    if r.status_code == 200:
        nutrition_ninja.extend(r.json())
    sleep(1)

df_ninja = pd.DataFrame(nutrition_ninja)[
    ["name", "serving_size_g", "calories", "protein_g", "carbohydrates_total_g", "fat_total_g", "fiber_g"]
]

# === 2. Nutritionix API ===
nutritionix_records = []
for query in food_queries:
    r = requests.post(
        "https://trackapi.nutritionix.com/v2/natural/nutrients",
        headers={
            "x-app-id": NUTRITIONIX_APP_ID,
            "x-app-key": NUTRITIONIX_APP_KEY,
            "Content-Type": "application/json"
        },
        json={"query": query}
    )
    if r.status_code == 200:
        for f in r.json().get("foods", []):
            nutritionix_records.append({
                "name": f["food_name"],
                "serving_size_g": f.get("serving_weight_grams"),
                "calories": f.get("nf_calories"),
                "protein_g": f.get("nf_protein"),
                "carbohydrates_total_g": f.get("nf_total_carbohydrate"),
                "fat_total_g": f.get("nf_total_fat"),
                "fiber_g": f.get("nf_dietary_fiber")
            })
    sleep(1)

df_nutritionix = pd.DataFrame(nutritionix_records)

# === Combine Nutrition Data ===
df_nutrition = pd.concat([df_ninja, df_nutritionix], ignore_index=True).drop_duplicates().head(100)

# === 3. TheMealDB Recipes & Ingredients ===
recipe_list = []
ingredient_list = []

for keyword in ["chicken", "beef", "rice", "vegetable", "egg", "pasta"]:
    r = requests.get(f"https://www.themealdb.com/api/json/v1/1/search.php?s={keyword}")
    meals = r.json().get("meals", [])
    for meal in meals:
        recipe_list.append({
            "title": meal.get("strMeal"),
            "description": f"{meal.get('strMeal')} from {meal.get('strArea')} - {meal.get('strCategory')}",
            "servings": 1,
            "cook_time_minutes": None,
            "ingredients": "; ".join([
                f"{meal.get(f'strMeasure{i}', '').strip()} {meal.get(f'strIngredient{i}', '').strip()}"
                for i in range(1, 21)
                if meal.get(f'strIngredient{i}') and meal.get(f'strIngredient{i}').strip()
            ]),
            "instructions": meal.get("strInstructions"),
            "nutrition": meal.get("nutrition"),  # May be None or missing
            "tags": meal.get("strTags")
        })

        for i in range(1, 21):
            ing = meal.get(f"strIngredient{i}")
            qty = meal.get(f"strMeasure{i}")
            if ing and ing.strip():
                ingredient_list.append({
                    "name": ing.strip(),
                    "quantity": qty.strip() if qty else "",
                    "category": meal.get("strCategory")
                })

    sleep(1)

df_recipe = pd.DataFrame(recipe_list).drop_duplicates().head(100)
df_ingredient = pd.DataFrame(ingredient_list).drop_duplicates().head(100)

# === Clean Recipe Data ===
df_recipe["nutrition"] = df_recipe["nutrition"].apply(
    lambda x: np.random.randint(1, 1001) if pd.isna(x) or str(x).strip() == "" else x
)
df_recipe.dropna(subset=["title", "ingredients", "instructions"], inplace=True)
df_recipe = df_recipe[
    (df_recipe["title"].str.strip() != "") &
    (df_recipe["ingredients"].str.strip() != "") &
    (df_recipe["instructions"].str.strip() != "")
].reset_index(drop=True)

df_nutrition.to_excel('nutrition_data_final.xlsx', sheet_name="NutritionInfo", index=False)
print("File saved: nutrition_data_final.xlsx")
df_ingredient.to_excel('ingredient_data_final.xlsx', sheet_name="Ingredient", index=False)
print("File saved: ingredient_data_final.xlsx")
df_recipe.to_excel('recipe_data_final.xlsx', sheet_name="Recipe", index=False)
print("File saved: recipe_data_final.xlsx")